In [1]:
import subprocess
import sys
from moviepy.editor import *
import speech_recognition as sr 
import cv2
from googletrans import Translator
from pydub import AudioSegment
import simpleaudio as sa
from scipy.io import wavfile
import noisereduce as nr

In [2]:
# get length of video file

def get_duration_of_mp4(filename):
    clip = VideoFileClip(filename)
    duration = clip.duration
    return duration

# extract aufio from vide
def extract_audio_from_mp4_to_wav(filename, path_to_safe_audio, start_time, end_time):
    video = VideoFileClip(filename).subclip(start_time, end_time)
    audio = video.audio
    audio.write_audiofile(path_to_safe_audio)
    return audio

def recognize_speech_from_file(audio_file):
    # create a speech recognition object
    r = sr.Recognizer()
    
    # open the audio file
    audio = AudioSegment.from_file(audio_file)

    # filter out the background noise
    audio = audio.high_pass_filter(1000)

    # boost the volume of the audio
    audio = audio.apply_gain(20)

    # convert the audio to a wave file
    audio.export("filtered_audio.wav", format="wav")

    # open the filtered audio file
    with sr.AudioFile("filtered_audio.wav") as source:
        # read the audio file
        audio_data = r.record(source)

    # recognize the speech in the audio file
    russian_text = r.recognize_google(audio_data, language='ru')
    
    # create a translation object
    translator = Translator()

    # translate the text from Russian to English
    english_translation = translator.translate(russian_text, src='ru', dest='en').text
    print(english_translation)

def enhance_speech(audio_file):
    # create a speech recognition object
    r = sr.Recognizer()

    # open the audio file
    audio = AudioSegment.from_file(audio_file)

    # filter out the background noise
    audio = audio.high_pass_filter(1000)

    # boost the volume of the audio
    audio = audio.apply_gain(20)

    # save the enhanced audio to the "audio" subdirectory
    audio.export("audio/enhanced_audio.wav", format="wav")

def play_audio(audio_file):
    # open the audio file
    wave_obj = sa.WaveObject.from_wave_file(audio_file)

    # play the audio
    play_obj = wave_obj.play()
    play_obj.wait_done()

def remove_noise(input_data, output_data):
    # load data
    rate, data = wavfile.read(input_data)
    # perform noise reduction
    reduced_noise = nr.reduce_noise(y=data, sr=rate)
    wavfile.write(output_data, rate, reduced_noise)

remove_noise('audio/audio0_till_60.wav', 'audio/enhanced_audio.wav')


In [29]:
duration = get_duration_of_mp4('Wedding.mp4')
print(duration)
parts = duration/(60)

parts = int(parts)
parts = list(range(parts))
subtitles = ''
for i in parts:
    if i == 0:
        start_time = 0
        end_time = start_time + 60
    elif i != parts[-1]:
        start_time = end_time + 1
        end_time = start_time + 60
    else:
        start_time = end_time + 1
        end_time = duration
    path_to_safe_audio = f'audio/audio_{start_time}_till_{end_time}.wav'
    extract_audio_from_mp4_to_wav(filename='Wedding.mp4', path_to_safe_audio=path_to_safe_audio, start_time=start_time, end_time=end_time)
    print(i,':', start_time, '-', end_time)
    enhance_speech(path_to_safe_audio)
    play_audio('audio/enhanced_audio.wav')
    #subtitles_of_part = recognize_speech_from_file(path_to_safe_audio)
    #subtitles += subtitles_of_part
    
    #print(subtitles_of_part) 
    
print(subtitles)
    

    

18136.06
MoviePy - Writing audio in audio/audio0_till_60.wav


MoviePy - Done.
0 : 0 - 60


AttributeError: 'array.array' object has no attribute 'shape'